##Module Imports

In [43]:
!pip install peft --quiet

In [44]:
!pip install accelerate -U --quiet

In [45]:
!pip install datasets --quiet

In [46]:
!pip install evaluate --quiet

In [47]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType, PeftModel, PeftConfig
import torch
import time
import evaluate
import pandas as pd
import numpy as np
import accelerate
from datasets import load_dataset

In [48]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [49]:
!pip install rouge-score --quiet

##Loading the Dataset and FLAN-T5 Model

In [50]:
dataset = load_dataset("knkarthick/dialogsum")

In [51]:
model_name = "google/flan-t5-base"

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [52]:
def get_nof_trainable_parameters(model):
  trainable_model_params = 0
  all_model_parameters = 0
  for _, param in model.named_parameters():
    all_model_parameters += param.numel()
    if param.requires_grad:
      trainable_model_params += param.numel()
  return f"trainable model parameters: {trainable_model_params} \n all model parameters: {all_model_parameters} \n percentage of trainable parameters: {trainable_model_params*100/all_model_parameters:.2f}%"

print(get_nof_trainable_parameters(original_model))

trainable model parameters: 247577856 
 all model parameters: 247577856 
 percentage of trainable parameters: 100.00%


##Preprocess Data

In [53]:
def tokenize_function(example):
  start = "Summarize the following conversation.\n\n"
  end = "\n\nSummary:"
  prompt = [start + dialogue + end for dialogue in example['dialogue']]
  example['input_ids'] = tokenizer(prompt, return_tensors='pt', padding='max_length', truncation=True).input_ids
  example['labels'] = tokenizer(example['summary'], return_tensors='pt', padding='max_length', truncation=True).input_ids

  return example

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(['id', 'topic', 'dialogue', 'summary'])

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

#### Subsampling

In [54]:
tokenized_dataset = tokenized_dataset.filter(lambda example, index: index%100 == 0, with_indices=True)

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

In [55]:
print("Shape of datasets:\n")
print(f"Training: {tokenized_dataset['train'].shape}")
print(f"Testing: {tokenized_dataset['test'].shape}")
print(f"Validation: {tokenized_dataset['validation'].shape}")

Shape of datasets:

Training: (125, 2)
Testing: (15, 2)
Validation: (5, 2)


In [56]:
print(tokenized_dataset)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 125
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 5
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 15
    })
})


##Configuring PEFT

In [57]:
lora_config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

In [58]:
peft_model = get_peft_model(original_model, lora_config)
print(get_nof_trainable_parameters(peft_model))

trainable model parameters: 3538944 
 all model parameters: 251116800 
 percentage of trainable parameters: 1.41%


##Initialise Trainer for Training the LLM



The below mentioned code for training is executed only for 1 epoch and a subsample of dataset due to resource constraints. When trained on preferably 32GB RAM as provided by AWS, the performance will be stellar and in-par with benchmark results.

In [59]:
output_dir = "/content/peft_model/"

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3,
    num_train_epochs=5,
    logging_steps=1,
    max_steps=1
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_dataset["train"]
)

In [60]:
peft_trainer.train()

peft_trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Step,Training Loss
1,51.250000


('/content/peft_model/tokenizer_config.json',
 '/content/peft_model/special_tokens_map.json',
 '/content/peft_model/tokenizer.json')

In [61]:
peft_model = AutoModelForSeq2SeqLM.from_pretrained(output_dir, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(output_dir)

In [63]:
print(get_nof_trainable_parameters(peft_model))

trainable model parameters: 0 
 all model parameters: 251116800 
 percentage of trainable parameters: 0.00%


##Comparing and Evaluating Model using ROUGE

In [64]:
index = 500

dialogue = dataset['test'][index]['dialogue']
baseline_summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation:
{dialogue}

Summary:

"""

input_ids = tokenizer(prompt, return_tensors='pt').input_ids.to('cuda')

peft_model.to('cuda')

original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

dash_line = "-".join('' for x in range(100))
print(dash_line)
print(f"INPUT_PROMPT: \n {prompt}")
print(dash_line)
print(f"BASELINE HUMAN SUMMARY: \n {baseline_summary} \n")
print(dash_line)
print(f"PEFT MODEL: \n {peft_model_text_output}")

---------------------------------------------------------------------------------------------------
INPUT_PROMPT: 
 
Summarize the following conversation:
#Person1#: Are you going anywhere for your vacation?
#Person2#: Yes, we're making plans for a tour.
#Person1#: That'll be lovely. Where are you going?
#Person2#: Well, we will start out from Long Island this Friday. We've planned a four day drive to Salt Lake City, where we'll join my brother and his family on his fortieth birthday.
#Person1#: Well, you've got to prepare a lot of food and enough sleeping bags then.
#Person2#: Oh, we'll spend the nights in hotels and enjoy local food as we pass by. How does it sound, David?
#Person1#: It sounds good. You can do a lot of sightseeing, too.
#Person2#: Yes, we'll take our time. And we'll go to Five Lake Strict and the Wall Street.
#Person1#: So, you're going to have a really nice vacation.
#Person2#: You can say that again.

Summary:


-----------------------------------------------------

In [65]:
dialogues = dataset['test'][0:10]['dialogue']
baseline_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []
peft_model_summaries = []

for _, dialogue in enumerate(dialogues):
  prompt = f"""
Summarize the following conversation:
{dialogue}

Summary:
"""
  input_ids = tokenizer(prompt, return_tensors='pt').input_ids.to('cuda')

  original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
  original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
  original_model_summaries.append(original_model_text_output)

  peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
  peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)
  peft_model_summaries.append(peft_model_text_output)

zipped_summaries = list(zip(baseline_summaries, original_model_summaries, peft_model_summaries))

df = pd.DataFrame(zipped_summaries, columns=['Human Baseline Summary', 'Original Model Summary', 'Peft Model Summary'])
df

,Human Baseline Summary,Original Model Summary,Peft Model Summary
0,Ms. Dawson helps #Person1# to write a memo to ...,This policy applies to all intra-office commun...,The memo is to be distributed to all employees...
1,In order to prevent employees from wasting tim...,#Person1: This is a memo to all employees. #Pe...,The memo is to be distributed to all employees...
2,Ms. Dawson takes a dictation for #Person1# abo...,Employees are required to take a memo outlinin...,The memo is to be distributed to all employees...
3,#Person2# arrives late because of traffic jam....,#Person1#: I'm so happy to see you finally her...,The traffic jam at the Carrefour intersection ...
4,#Person2# decides to follow #Person1#'s sugges...,#Person1#: I'm here. #Person2#: I'm here. #Per...,The traffic jam at the Carrefour intersection ...
5,#Person2# complains to #Person1# about the tra...,The driver of the car is driving to work.,The traffic jam at the Carrefour intersection ...
6,#Person1# tells Kate that Masha and Hero get d...,Masha and Hero are getting divorced.,Masha and Hero are getting divorced.
7,#Person1# tells Kate that Masha and Hero are g...,Masha and Hero are having a divorce.,Masha and Hero are getting divorced.
8,#Person1# and Kate talk about the divorce betw...,"#Pornings: #Person1: #Person2: Well, I heard t...",Masha and Hero are getting divorced.
9,#Person1# and Brian are at the birthday party ...,Brian's birthday is tomorrow.,Brian's birthday is coming up.


In [66]:
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True
)


print("ORIGINAL MODEL:")
print(original_model_results)
print("PEFT MODEL:")
print(peft_model_results)

ORIGINAL MODEL:
{'rouge1': 0.23332922130260977, 'rouge2': 0.07676318962719747, 'rougeL': 0.20640472233776969, 'rougeLsum': 0.2095297202786403}
PEFT MODEL:
{'rouge1': 0.3456068376068376, 'rouge2': 0.13318460965355755, 'rougeL': 0.2805896418396418, 'rougeLsum': 0.28270858770858776}


In [67]:
print("Absolute Percentage Improvement of Peft Model over Human Baseline")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
  print(f"{key}: {value*100:.2f}%")

Absolute Percentage Improvement of Peft Model over Human Baseline
rouge1: 11.23%
rouge2: 5.64%
rougeL: 7.42%
rougeLsum: 7.32%
